<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- Download sample notebook from [here](https://github.com/pmservice/ai-openscale-tutorials/blob/master/notebooks/Credit%20%20model%20with%20SageMaker%20linear-learner%20.ipynb)
- Run the notebook to train a model and create deployment endpoint

### 1.1 Installation and authentication

In [1]:
!pip install sagemaker --no-cache | tail -n 1

In [2]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

**Action:** Restart the kernel.

Import and initiate.

In [3]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```
bx target -g <myResourceGroup>
```
- get details of the instance
```
bx resource service-instance 'AI-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [4]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [7]:
# The code was removed by Watson Studio for sharing.

In [12]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

Create schema for data mart.

In [13]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [14]:
client = APIClient(AIOS_CREDENTIALS)

In [15]:
client.version

'1.0.287'

### 1.2 DataMart setup

In [17]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [18]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [19]:
SAGEMAKER_ENGINE_CREDENTIALS = {
        "access_key_id": "***",
        "secret_access_key": "***",
        "region": "***"
}

In [20]:
# The code was removed by Watson Studio for sharing.

In [21]:
binding_uid = client.data_mart.bindings.add('My SageMaker engine', SageMakerMachineLearningInstance(SAGEMAKER_ENGINE_CREDENTIALS))

In [22]:
bindings_details = client.data_mart.bindings.get_details()

In [23]:
client.data_mart.bindings.list()

e407d23c-dad5-4675-bb05-05180c6424f1,My SageMaker engine,amazon_sagemaker,2018-12-18T14:48:50.750Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [24]:
client.data_mart.bindings.list_assets()

e506b314bfec52b83800b2696372045b,Breast-cancer-linear-learner-2018-12-17-16-11-17,2018-12-17T16:27:15.010Z,model,,e407d23c-dad5-4675-bb05-05180c6424f1,False
b43f50a20656d2c886e0d2410dc57323,factorization-machines-2018-10-18-12-58-24-328,2018-10-18T12:58:24.456Z,model,,e407d23c-dad5-4675-bb05-05180c6424f1,False
0530ab0cd4f4dd5486b19c08df8b6914,DEMO-multi-classification-2018-10-10-14-26-26,2018-10-10T14:31:44.348Z,model,,e407d23c-dad5-4675-bb05-05180c6424f1,False
85aba02937c71f90ae3db6d98214766d,DEMO-classification-2018-10-10-13-24-25,2018-10-10T13:29:38.548Z,model,,e407d23c-dad5-4675-bb05-05180c6424f1,False


**Action:** Assign your source_uid to `source_uid` variable below.

In [25]:
source_uid = '0530ab0cd4f4dd5486b19c08df8b6914'

In [26]:
subscription = client.data_mart.subscriptions.add(
    SageMakerMachineLearningAsset(
                source_uid=source_uid,
                binding_uid=binding_uid,
                input_data_type=InputDataType.STRUCTURED,
                problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
                label_column='diagnosis',
                probability_column='score',
                prediction_column='predicted_label'))

#### Get subscriptions list

In [27]:
subscriptions = client.data_mart.subscriptions.get_details()

In [28]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['0530ab0cd4f4dd5486b19c08df8b6914']


#### List subscriptions

In [29]:
client.data_mart.subscriptions.list()

0530ab0cd4f4dd5486b19c08df8b6914,DEMO-multi-classification-2018-10-10-14-26-26,model,e407d23c-dad5-4675-bb05-05180c6424f1,2018-12-18T14:48:58.002Z


<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the product line model and measure response time

In [37]:
import requests
import time
import json
import boto3

binding_details = client.data_mart.bindings.get_details(binding_uid=binding_uid)
subscription_details = subscription.get_details()

access_id = binding_details['entity']['credentials']['access_key_id']
access_key = binding_details['entity']['credentials']['secret_access_key']
region = binding_details['entity']['credentials']['region']
endpoint_name = subscription_details['entity']['deployments'][0]['name']
runtime = boto3.client('sagemaker-runtime', region_name=region, aws_access_key_id=access_id, aws_secret_access_key=access_key)

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
        
payload = "17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482\n17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482"

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": [
        5.532644564709699e-09,
        1.0
      ],
      "predicted_label": 1.0
    },
    {
      "score": [
        5.532644564709699e-09,
        1.0
      ],
      "predicted_label": 1.0
    }
  ]
}


### 4.2 Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [38]:
values = []

for v in payload.split('\n'):
    values.append([float(s) for s in v.split(',')])

request_data = {'fields': fields, 'values': values}

response_data = {'fields': list(result['predictions'][0]),
            'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [39]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

#### Store the payload using REST API

Get the token first.

In [40]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":AIOS_CREDENTIALS["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

Store the payload.

In [41]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(AIOS_CREDENTIALS['url'], AIOS_CREDENTIALS['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

Request OK: True


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [46]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=5)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [43]:
subscription.feedback_logging.print_table_schema()

radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True
concave points_mean,double,True
symmetry_mean,double,True
fractal_dimension_mean,double,True
radius_se,double,True


The feedback records can be send to feedback table using below code.

In [44]:
feedback_records = []

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']

for i in range(1, 10):
    feedback_records.append([17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482, 1])

subscription.feedback_logging.store(feedback_data=feedback_records, fields=fields)

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [47]:
run_details = subscription.quality_monitoring.run()

Since the monitoring runs in the background you can use below method to check the status of the job.

In [48]:
status = run_details['status']
id = run_details['id']

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

Run status: initializing
Run status: completed


### Show the quality metrics

In [49]:
subscription.quality_monitoring.show_table()

2018-12-18 14:55:14.589000+00:00,1.0,0.7,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,Accuracy_evaluation_7e20be61-a3a4-40ae-ae9e-b1ad3631ddc2,


Get all calculated metrics.

In [50]:
deployment_uids = subscription.get_deployment_uids()

In [51]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

{'end': '2018-12-18T14:55:54.066335Z',
 'metrics': [{'process': 'Accuracy_evaluation_7e20be61-a3a4-40ae-ae9e-b1ad3631ddc2',
   'timestamp': '2018-12-18T14:55:14.589Z',
   'value': {'metrics': [{'name': 'weightedTruePositiveRate', 'value': 1.0},
     {'name': 'accuracy', 'value': 1.0},
     {'name': 'weightedFMeasure', 'value': 1.0},
     {'name': 'weightedRecall', 'value': 1.0},
     {'name': 'weightedFalsePositiveRate', 'value': None},
     {'name': 'weightedPrecision', 'value': 1.0}],
    'quality': 1.0,
    'threshold': 0.7}}],
 'start': '2018-12-18T13:53:17.330Z'}

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Print schema of payload_logging table

In [52]:
subscription.payload_logging.print_table_schema()

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True


#### Show (preview) the table

In [53]:
subscription.payload_logging.describe_table()

        radius_mean  texture_mean  perimeter_mean     area_mean  \
count  4.600000e+01  4.600000e+01    4.600000e+01  4.600000e+01   
mean   1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   
std    1.077591e-14  1.436789e-14    8.620731e-14  2.298862e-13   
min    1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   
25%    1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   
50%    1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   
75%    1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   
max    1.702000e+01  2.398000e+01    1.128000e+02  8.993000e+02   

       smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
count     4.600000e+01      4.600000e+01    4.600000e+01         4.600000e+01   
mean      1.197000e-01      1.496000e-01    2.417000e-01         1.203000e-01   
std       7.015569e-17      1.403114e-16    1.683737e-16         4.209341e-17   
min       1.197000e-01      1.496000e-01    2.417000e-01         1.20300

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,predicted_label
count,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,...,4.600000e+01,4.600000e+01,46.0,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,4.600000e+01,46.0
mean,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0
std,1.077591e-14,1.436789e-14,8.620731e-14,2.298862e-13,7.015569e-17,1.403114e-16,1.683737e-16,4.209341e-17,8.418683e-17,7.015569e-17,...,1.436789e-14,8.620731e-14,0.0,1.683737e-16,5.612455e-17,4.489964e-16,8.418683e-17,1.683737e-16,8.418683e-17,0.0
min,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0
25%,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0
50%,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0
75%,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0
max,1.702000e+01,2.398000e+01,1.128000e+02,8.993000e+02,1.197000e-01,1.496000e-01,2.417000e-01,1.203000e-01,2.248000e-01,6.382000e-02,...,3.209000e+01,1.361000e+02,1344.0,1.634000e-01,3.559000e-01,5.588000e-01,1.847000e-01,3.530000e-01,8.482000e-02,1.0


#### Return the table content as pandas dataframe

In [54]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feddback logging

Check the schema of table.

In [55]:
subscription.feedback_logging.print_table_schema()

radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True
concave points_mean,double,True
symmetry_mean,double,True
fractal_dimension_mean,double,True
radius_se,double,True


Preview table content.

In [56]:
subscription.feedback_logging.show_table()

17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2018-12-18 14:54:46.373000+00:00


Describe table (calulcate basic statistics).

In [57]:
subscription.feedback_logging.describe_table()

       radius_mean  texture_mean  perimeter_mean     area_mean  \
count         9.00  9.000000e+00    9.000000e+00  9.000000e+00   
mean         17.02  2.398000e+01    1.128000e+02  8.993000e+02   
std           0.00  3.768222e-15    1.507289e-14  1.205831e-13   
min          17.02  2.398000e+01    1.128000e+02  8.993000e+02   
25%          17.02  2.398000e+01    1.128000e+02  8.993000e+02   
50%          17.02  2.398000e+01    1.128000e+02  8.993000e+02   
75%          17.02  2.398000e+01    1.128000e+02  8.993000e+02   
max          17.02  2.398000e+01    1.128000e+02  8.993000e+02   

       smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
count     9.000000e+00      9.000000e+00          9.0000               9.0000   
mean      1.197000e-01      1.496000e-01          0.2417               0.1203   
std       1.471962e-17      2.943923e-17          0.0000               0.0000   
min       1.197000e-01      1.496000e-01          0.2417               0.1203   


Get table content.

In [58]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [59]:
subscription.quality_monitoring.print_table_schema()

ts,timestamp,False
quality,float,False
quality_threshold,float,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [60]:
subscription.quality_monitoring.show_table()

2018-12-18 14:55:14.589000+00:00,1.0,0.7,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,Accuracy_evaluation_7e20be61-a3a4-40ae-ae9e-b1ad3631ddc2,


### 6.4 Performance metrics table

In [61]:
subscription.performance_monitoring.print_table_schema()

ts,timestamp,False
scoring_time,float,False
scoring_records,object,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [62]:
subscription.performance_monitoring.show_table()

2018-12-18 14:52:32.576601+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576707+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576642+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576739+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576624+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576658+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576723+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576755+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576536+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,
2018-12-18 14:52:32.576675+00:00,0.0,2,e407d23c-dad5-4675-bb05-05180c6424f1,0530ab0cd4f4dd5486b19c08df8b6914,37a83f399e6dc3b9d08d7d01fe690665,,


Note: First 10 records were displayed.


### 6.5 Data Mart measurement facts table

In [63]:
client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'asset': {'asset_id': '0530ab0cd4f4dd5486b19c08df8b6914',
    'asset_type': 'model',
    'created_at': '2018-10-10T14:31:44.348Z',
    'name': 'DEMO-multi-classification-2018-10-10-14-26-26',
    'url': 's3://sagemaker-us-east-1-014862798213/sagemaker/DEMO-breast-cancer-prediction/DEMO-multi-classification-2018-10-10-14-26-26/output/model.tar.gz'},
   'deployment': {'created_at': '2018-10-10T14:39:21.421Z',
    'deployment_id': '37a83f399e6dc3b9d08d7d01fe690665',
    'deployment_type': 'online',
    'name': 'DEMO-multi-classification-endpoint-201810101439',
    'url': 'DEMO-multi-classification-endpoint-201810101439'},
   'metrics': [{'issues': 0,
     'metric_type': 'performance',
     'timestamp': '2018-12-18T14:53:10.123340Z',
     'value': {'records': 2, 'response_time': 250.0}},
    {'issues': 0,
     'metric_type': 'quality',
     'timestamp': '2018-12-18T14:55:14.589Z',
     'value': {'metrics': [{'name': 'weightedTruePositiveRate', 'value': 1.0},
      

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.